Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [17]:
batch_size = 128
beta = .0001
learn_rate = 0.1

hidden_1_units = 128
hidden_2_units = 64
hidden_3_units = 32

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    data_placeholder = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    labels_placeholder = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder(tf.float32)
    

    def inference(data):
        with tf.name_scope('hidden_1'):
            weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_1_units], 
                                stddev=1.0 / math.sqrt(hidden_1_units)))
            biases = tf.Variable(tf.zeros([hidden_1_units]))
            z = tf.matmul(data, weights) + biases
            hidden_1 = tf.nn.relu(z)
            
        with tf.name_scope('hidden_2'):
            h_fc1_drop = tf.nn.dropout(hidden_1, keep_prob)
            weights = tf.Variable(
            tf.truncated_normal([hidden_1_units, hidden_2_units], 
                                stddev=1.0 / math.sqrt(hidden_2_units)))
            biases = tf.Variable(tf.zeros([hidden_2_units]))
            z = tf.matmul(h_fc1_drop, weights) + biases
            hidden_2 = tf.nn.relu(z)
            
        with tf.name_scope('hidden_3'):
            h_fc1_drop = tf.nn.dropout(hidden_2, keep_prob)
            weights = tf.Variable(
            tf.truncated_normal([hidden_2_units, hidden_3_units], 
                                stddev=1.0 / math.sqrt(hidden_3_units)))
            biases = tf.Variable(tf.zeros([hidden_3_units]))
            z = tf.matmul(h_fc1_drop, weights) + biases
            hidden_3 = tf.nn.relu(z)

        with tf.name_scope('linear'):
            # Variables.
            h_fc1_drop = tf.nn.dropout(hidden_3, keep_prob)
            weights = tf.Variable(tf.truncated_normal([hidden_3_units, num_labels]))
            biases = tf.Variable(tf.zeros([num_labels]))
            logits = tf.matmul(h_fc1_drop, weights) + biases
        
        return logits

    def loss(logits, labels):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, labels)
        loss = tf.reduce_mean(cross_entropy)
        loss_prime = loss + beta * tf.nn.l2_loss(logits)
        
        return loss_prime
    
    def evaluation(logits, labels):
        correct =  tf.nn.in_top_k(logits, tf.argmax(labels, 1), 1)
        return tf.reduce_sum(tf.cast(correct, tf.int32))

    # Training computation.
    logits = inference(data_placeholder)
    L = loss(logits, labels_placeholder)

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learn_rate, global_step, 5000, .96)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(L, global_step=global_step)

    # Predictions for the training, validation, and test data.    
    eval_correct = evaluation(logits, labels_placeholder)

In [18]:
def do_eval(sess, data, labels):
    steps = len(labels) // batch_size
    correct = 0
    for step in range(steps):
        offset = (step * batch_size) % (labels.shape[0] - batch_size)
        batch_data = data[offset : (offset + batch_size), :]
        batch_label = labels[offset : (offset + batch_size), :]
        feed_dict = {data_placeholder : batch_data, labels_placeholder : batch_labels, keep_prob : 1}
        correct += sess.run(eval_correct, feed_dict=feed_dict)
        
    print("  Accuracy: {:.1f} {} / {}".format((correct / len(labels)), correct, len(labels)))

In [19]:
num_steps = 2000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {data_placeholder : batch_data, labels_placeholder : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, L, logits], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation: ")
            do_eval(session, valid_dataset, valid_labels)

    print("Test: ")
    do_eval(session, test_dataset, test_labels)
    
session.close()

Initialized
Minibatch loss at step 0: 11.959784
Minibatch accuracy: 13.3%
Validation: 
  Accuracy: 0.0 1037 / 10000
Minibatch loss at step 500: 1.571151
Minibatch accuracy: 56.2%
Validation: 
  Accuracy: 0.0 996 / 10000
Minibatch loss at step 1000: 1.250730
Minibatch accuracy: 72.7%
Validation: 
  Accuracy: 0.0 968 / 10000
Minibatch loss at step 1500: 1.306945
Minibatch accuracy: 70.3%
Validation: 
  Accuracy: 0.0 996 / 10000
Minibatch loss at step 2000: 1.173868
Minibatch accuracy: 73.4%
Validation: 
  Accuracy: 0.0 1029 / 10000
Minibatch loss at step 2500: 1.051755
Minibatch accuracy: 76.6%
Validation: 
  Accuracy: 0.0 941 / 10000
Minibatch loss at step 3000: 1.171562
Minibatch accuracy: 76.6%
Validation: 
  Accuracy: 0.0 998 / 10000
Minibatch loss at step 3500: 1.149547
Minibatch accuracy: 75.8%
Validation: 
  Accuracy: 0.0 991 / 10000


KeyboardInterrupt: 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
